# Design the biasing circuit
## Generate Vb1, Vss, Vbias
- Given Ibias = 38.34uA
- Vb1 = 0.925V
- Vss = 0.564V
- Vbias = 0.48V

In [1]:
# Always run this section first
import numpy as np
import matplotlib.pyplot as plt
from look_up import *
nch_2v = importdata("nch_2v.mat")
pch_2v = importdata("pch_2v.mat")
nch_1v = importdata("nch_1v.mat")
pch_1v = importdata("pch_1v.mat")

- look up the ro under different bias ID, since the ro is not enough for my first design

In [3]:
# upper MOSFET
VSD1 = 1.8 - 0.986418
VSG1 = 1.8 - 0.975
VSB1 = 0
L = 0.36
ID = [4e-6, 5e-6, 6e-6, 7e-6, 8e-6, 9e-6, 10e-6]
gds_ID1 = look_up_basic(pch_1v, 'GDS_ID', vgs = VSG1, vds = VSD1, vsb=VSB1, l=L)
for i in range(len(ID)): 
    ro1 = 1 / (ID[i]*gds_ID1)
    print("ro1: ", ro1)

# lower MOSFET
VDS2 = 0.506418
VGS2 = 0.564
VSB2 = 0
L = 0.36
gds_ID2 = look_up_basic(nch_1v, 'GDS_ID', vgs = VGS2, vds = VDS2, vsb=VSB2, l=L)
for i in range(len(ID)):
    ro2 = 1 / (ID[i]*gds_ID2)
    print("ro2: ", ro2)



ro1:  1501545.5161831165
ro1:  1201236.412946493
ro1:  1001030.3441220776
ro1:  858026.009247495
ro1:  750772.7580915582
ro1:  667353.5627480516
ro1:  600618.2064732464
ro2:  674564.9394097764
ro2:  539651.951527821
ro2:  449709.9596065176
ro2:  385465.6796627294
ro2:  337282.4697048882
ro2:  299806.63973767834
ro2:  269825.9757639105


## M1

In [5]:
# diode connection
VDD = 1.8
Vb1 = 0.975
VSG1 = VDD - Vb1
VSD1 = VSG1
VSB1 = 0
L = 0.36
ID1 = 38.34e-6

# calculate the width of M1
JD1 = look_up_basic(pch_1v, 'ID', vgs = VSG1, vds = VSD1, vsb=VSB1, l=L)
W1 = ID1/JD1
print("W1: ", W1)

W1:  0.5700682424122965


## M2

In [6]:
# Current mirror output
VDD = 1.8
Vb1 = 0.975
Vb_up = 0.986418
VSG2 = VDD - Vb1
VSD2 = VDD - Vb_up
VSB2 = 0
L = 0.36
ID2 = 38.34e-6

# calculate the width of M2
JD2 = look_up_basic(pch_1v, 'ID', vgs = VSG2, vds = VSD2, vsb=VSB2, l=L)
W2 = ID2/JD2
print("W2: ", W2)


W2:  0.5711526546053701


# The previous version is not qualifying, since RL is not enough
## The second version

In [28]:
# cascaded current mirror
L = 0.36
V1 = 0.25
VDD = 1.8
Vout1 = 0.506418
Vout2 = Vout1 + 0.48

W4 = 0.931*2 # in case the ratio of different currents are bigger than 10:1
VGS4 = 0.564
VDS4 = V1
VSB4 = 0

# calculate the current
JD4 = look_up_basic(nch_2v, 'ID', vgs = VGS4, vds = VDS4, vsb=VSB4, l=L)
ID  = JD4*W4
print("ID: ", ID)

# calculate the ro4
gds_ID4 = look_up_basic(nch_2v, 'GDS_ID', vgs = VGS4, vds = VDS4, vsb=VSB4, l=L)
ro4 = 1 / (ID*gds_ID4)
print("ro4: ", ro4)

# find VGS6
W6 = W4
JD6 = JD4
VGS6_set = np.linspace(0, 1.8, 1000)
VSB6 = V1
VDS6 = Vout1 - V1
for i in range(len(VGS6_set)):
    VGS6 = VGS6_set[i]
    JD_tmp = look_up_basic(nch_2v, 'ID', vgs = VGS6, vds = VDS6, vsb=VSB6, l=L)
    if (JD_tmp > JD6):
        break

print("VGS6: ", VGS6)
gm_ID6 = look_up_basic(nch_2v, 'GM_ID', vgs = VGS6, vds = VDS6, vsb=VSB6, l=L)
gds_ID6 = look_up_basic(nch_2v, 'GDS_ID', vgs = VGS6, vds = VDS6, vsb=VSB6, l=L)
Av6 = gm_ID6 / gds_ID6
R_lower = Av6*ro4
print("R_lower: ", R_lower)

# Design the bias circuit
V2 = V1 + VGS6
print("V2: ", V2)
W5 = W6
W3 = W4
VGS3 = VGS4
VSB3 = 0
V3_set = np.linspace(0, 1.8, 1000)

# calculate the current
for i in range(len(V3_set)):
    V3 = V3_set[i]
    VDS3 = V3
    VSB5 = V3
    VGS5 = V2 - V3
    VDS5 = VGS5
    JD5 = look_up_basic(nch_2v, 'ID', vgs = VGS5, vds = VDS5, vsb=VSB5, l=L)
    JD3 = look_up_basic(nch_2v, 'ID', vgs = VGS3, vds = VDS3, vsb=VSB3, l=L)
    if (JD3 > JD5): # since the width of M3 and M5 are the same
        break

ID_bias = JD5*W5
print("V3: ", V3)
print("ID_bias: ", ID_bias)

# design the upper bias circuit
V4 = (VDD + V2) / 2
VSG9 = VDD - V4
VSD9 = VSG9
VSB9 = 0
VSG7 = V4 - V2
VSD7 = VSG7
VSB7 = 0
# calculate the width of M9 and M7
JD9 = look_up_basic(pch_2v, 'ID', vgs = VSG9, vds = VSD9, vsb=VSB9, l=L)
W9 = ID_bias/JD9
print("W9: ", W9)
JD7 = look_up_basic(pch_2v, 'ID', vgs = VSG7, vds = VSD7, vsb=VSB7, l=L)
W7 = ID_bias/JD7
print("W7: ", W7)

# design the upper output circuit, AKA M6 and M8
V5 = V4
VSG10 = VSG9
VSD10 = VDD - V5
VSB10 = 0
VSD8 = V5 - Vout2
VSB8 = 0
VSG8 = V5 - V2

# calculate the width of M6 and M8
JD10 = look_up_basic(pch_2v, 'ID', vgs = VSG10, vds = VSD10, vsb=VSB10, l=L)
JD8 = look_up_basic(pch_2v, 'ID', vgs = VSG8, vds = VSD8, vsb=VSB8, l=L)
W10 = ID / JD10
W8 = ID / JD8
print("W10: ", W10)
print("W8: ", W8)

# calculate the Rout of upper circuit
gds_ID10 = look_up_basic(pch_2v, 'GDS_ID', vgs = VSG10, vds = VSD10, vsb=VSB10, l=L)
gm_ID8 = look_up_basic(pch_2v, 'GM_ID', vgs = VSG8, vds = VSD8, vsb=VSB8, l=L)
gds_ID8 = look_up_basic(pch_2v, 'GDS_ID', vgs = VSG8, vds = VSD8, vsb=VSB8, l=L)

ro10 = 1 / (gds_ID10*ID)
Av8 = gm_ID8 / gds_ID8

# design the diode connected M10
VSG11 = 0.48
VSB11 = 0
VSD11 = VSG11
JD11 = look_up_basic(pch_2v, 'ID', vgs = VSG11, vds = VSD11, vsb=VSB11, l=L)
W11 = ID/JD11
print("W11: ", W11)

# calculate the total RL
gm_ID11 = look_up_basic(pch_2v, 'GM_ID', vgs = VSG11, vds = VSD11, vsb=VSB11, l=L)
R_upper = 1/(gm_ID11*ID) + ro10*Av8
print("R_upper: ", R_upper)
RL = 1/(1/R_upper + 1/R_lower)
print("RL: ", RL)

print("W3: ", W3)
print("W4: ", W4)
print("W5: ", W5)
print("W6: ", W6)
print("W7: ", W7)
print("W8: ", W8)
print("W9: ", W9)
print("W10: ", W10)
print("W11: ", W11)
print("ID: ", ID)
print("ID_bias: ", ID_bias)




ID:  6.850574054321321e-06
ro4:  159777.0539956279
VGS6:  0.5981981981981982
R_lower:  2592602.4375088657
V2:  0.8481981981981982
V3:  0.26306306306306304
ID_bias:  6.841829081955099e-06
W9:  8.107023487187913
W7:  8.107023487187938
W10:  8.117385584153293
W8:  8.773081917073764
W11:  7.249071492469277
R_upper:  10662855.923784168
RL:  2085521.715305817
W3:  1.862
W4:  1.862
W5:  1.862
W6:  1.862
W7:  8.107023487187938
W8:  8.773081917073764
W9:  8.107023487187913
W10:  8.117385584153293
W11:  7.249071492469277
ID:  6.850574054321321e-06
ID_bias:  6.841829081955099e-06
